In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go 
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit
from scipy.stats import chi2

# Lettura dei dati


In [2]:
# salvo i dati in un dataframe

tempi = [0, 5, 12, 21, 28, 36, 43, 50, 60, 72, 84, 97]
data = {}

for i in tempi:
    filename = f'./data/t{i}.TXT'
    read = pd.read_csv(filename, delimiter='\t', skiprows=19)
    λ = read.iloc[:, 0].to_numpy()
    A = read.iloc[:, 1].to_numpy()
    data[i] = pd.DataFrame({'λ': λ, 'A': A})

tempi = np.array(tempi)

# Rimuovo i primi 50 dati per tempo zero perchè abbiamo preso misure in più
min_length = min(len(data[t]['λ']) for t in tempi)
if 0 in data:
    data[0] = data[0].iloc[50:].reset_index(drop=True)

# Fit parabolici

<h3> Dati iniziali

In [3]:
colors = px.colors.sample_colorscale('Dense', [i / 11 for i in range(12)])
fig = go.Figure()

for idx, (key, df) in enumerate(data.items()):
    fig.add_trace(go.Scatter(x = df['λ'], y = df['A'], mode = 'lines', name = f't = {key} ns', 
                             line = dict(color = colors[idx])))

    # error_y=dict( 
    #     type='data', 
    #     array = sigmay, 
    #     color = colors[2 * i],
    #     thickness = 1.5, 
    #     width = 3, 
	# 	)

fig.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Tempi',
    template = 'plotly_white'
)

fig.show()

<h3> Accoppio le code

In [4]:
# accoppio le code e creo un dataframe con i dati aggiornati

minimi = np.array([df['A'].min() for df in data.values()])
min_minimo = min(minimi) 
offset = minimi - min_minimo  # Calculate the offset for each time point

# Initialize data_agg as a dictionary
data_agg = {}

for idx, i in enumerate(tempi):
    data_agg[i] = data[i].copy()  # Create a copy of the original dataframe
    data_agg[i]['A'] = data[i]['A'] - offset[idx]  # Adjust the 'A' values using the corresponding offset



In [5]:
fig_agg = go.Figure()

for idx, (key, df) in enumerate(data_agg.items()):
    fig_agg.add_trace(go.Scatter(x=df['λ'], y=df['A'], mode='lines', name=f't = {key} ns', 
                             line=dict(color=colors[idx])))

    # error_y=dict( 
    #     type='data', 
    #     array = sigmay, 
    #     color = colors[2 * i],
    #     thickness = 1.5, 
    #     width = 3, 
	# 	)

fig_agg.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Tempi',
    template = 'plotly_white'
)

fig_agg.show()

<h3> Sottraggo le curve e rappresento

In [6]:
# faccio la sottrazione tra le curve

data_diff = {}

for idx, i in enumerate(tempi):
    data_diff[i] = data_agg[i].copy()  # Create a copy of the original dataframe
    data_diff[i]['A'] = data_agg[0]['A'] - data_agg[i]['A'] 


In [7]:
fig_diff = go.Figure()

for idx, (key, df) in enumerate(data_diff.items()):
    fig_diff.add_trace(go.Scatter(x=df['λ'], y=df['A'], mode='lines', name=f't = {key} ns', 
                             line=dict(color=colors[idx])))

    # error_y=dict( 
    #     type='data', 
    #     array = sigmay, 
    #     color = colors[2 * i],
    #     thickness = 1.5, 
    #     width = 3, 
	# 	)

fig_diff.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Tempi',
    template = 'plotly_white'
)

fig_diff.show()

<h3> Definisco la parabola e trovo i massimi. Salvo i subset attorno ai massimi

In [8]:
# Definizione della parabola per il fit
def parabola(x, a, xV, yV):
    return a * (x - xV)**2 + yV

r = 5

# Trova i massimi delle curve sottratte e salva i subset attorno ai massimi
picchi = {}
for key, df in data_diff.items():
    if key == 0:
        continue
    max_index = df['A'].idxmax()
    start = max(0, max_index - r)
    end = min(len(df), max_index + r + 1)
    subset = df.iloc[start:end]
    picchi[key] = subset

In [9]:
σ_λ = 1.5 #(nm) dichiarato dal costruttore
σ_A_1 = 0.004

<h3> Faccio l'effettivo fit parabolico, e plotto dati iniziali, parabole e vertici

In [10]:
fit_results = []
yV_err = []

fig_parabolic_fit = go.Figure()

# Dati sottratti
for idx, (key, df) in enumerate(data_diff.items()):

    fig_parabolic_fit.add_trace(go.Scatter(
        x=df['λ'],
        y=df['A'],
        mode='lines',
        name=f'Tempo {key}',
        line=dict(dash='dot', color=colors[idx], width=2),
        showlegend=False
    ))

# picchi, fit parabolico e errori
for key, subset in picchi.items():

    if key == 0:
        continue
    x_data = subset['λ']
    y_data = subset['A']


    # -------------- ERRORI e FIT ----------------

    # iterazione 0
    popt, pcov = curve_fit(parabola, x_data, y_data, p0=[-1, x_data.mean(), y_data.max()])
    a, xV, yV = popt
    err_a, err_xV, err_yV = np.sqrt(np.diag(pcov))

    # iterazione 1
    dydl = np.abs(2 * a * (x_data - xV))
    σ_y_prop = dydl * σ_λ
    σ_y = np.sqrt(σ_y_prop ** 2 + σ_A_1 ** 2)

    popt, pcov = curve_fit(parabola, x_data, y_data, p0 = [a, xV, yV], sigma = σ_y, absolute_sigma = True)
    a, xV, yV = popt
    err_a, err_xV, err_yV = np.sqrt(np.diag(pcov))

   # ---------------------------------------


    x_fit = np.linspace(x_data.min(), x_data.max(), 100)
    y_fit = parabola(x_fit, *popt)

    yV_err.append(err_yV)

    # Aggiungi i risultati alla lista
    fit_results.append({
        'Tempo (ns)': key,
        'a': a,
        'err a': err_a,
        'xV': xV,
        'err xV': err_xV,
        'yV': yV,
        'err yV': err_yV
    })
    
    # Aggiungi il fit parabolico
    fig_parabolic_fit.add_trace(go.Scatter(
        x = x_fit,
        y = y_fit,
        mode = 'lines',
        name = f'Fit Tempo {key}',
        line = dict(color=colors[7]),
        showlegend = False
    ))

    # Aggiungi il punto del picco
    fig_parabolic_fit.add_trace(go.Scatter(
        x = [popt[1]],  # xV
        y = [popt[2]],  # yV
        mode = 'markers',
        name = f'Picco Tempo {key}',
        marker = dict(size = 3, color = colors[10], symbol = 'circle'),
        error_y = dict(
            type = 'data',
            array = [err_yV],
            visible = True,
            color = colors[10],
            thickness = 1.5,
            width = 3
        ),
        # error_x = dict(
        #     type = 'data',
        #     array = [err_xV],
        #     visible = True,
        #     color = colors[10],
        #     thickness = 1.5,
        #     width = 3
        # ),
        showlegend = False
    ))

fig_parabolic_fit.update_layout(
    xaxis_title='λ',
    yaxis_title='A',
    legend_title='Legenda',
    template='plotly_white'
)

fig_parabolic_fit.show()


<h3> Stampo i parametri ricavati dai fit parabolici con i relativi errori

In [11]:
# Creazione del DataFrame
fit_results_df = pd.DataFrame(fit_results)

# Stampa della tabella
print(fit_results_df.round(5))

    Tempo (ns)        a    err a         xV   err xV       yV   err yV
0            5 -0.00007  0.00014  527.68231  2.83174  0.01728  0.00183
1           12 -0.00011  0.00014  526.29888  2.44151  0.03355  0.00176
2           21 -0.00013  0.00015  525.52788  1.63930  0.04549  0.00184
3           28 -0.00016  0.00015  525.95705  1.54099  0.05204  0.00182
4           36 -0.00020  0.00016  525.99843  1.33363  0.06400  0.00184
5           43 -0.00029  0.00017  526.34175  0.91791  0.07436  0.00194
6           50 -0.00032  0.00018  526.56290  0.84888  0.08287  0.00197
7           60 -0.00024  0.00016  526.52503  1.30940  0.08468  0.00182
8           72 -0.00032  0.00018  526.60459  0.84796  0.09356  0.00197
9           84 -0.00035  0.00019  526.49649  0.82578  0.10043  0.00199
10          97 -0.00031  0.00018  526.39169  1.04879  0.11013  0.00189


<h3> Chi quadro e pvalue per ogni fit

In [12]:
chi_squared_results = []

for key, subset in picchi.items():
    if key == 0:
        continue

    x_data = subset['λ']
    y_data = subset['A']

    # Fit (puoi saltarlo se hai salvato già i popt per ogni key, ma attenzione a non usare quelli dell’ultimo ciclo)
    popt, _ = curve_fit(parabola, x_data, y_data, p0=[-1, x_data.mean(), y_data.max()])
    a, xV, yV = popt
    dydl = np.abs(2 * a * (x_data - xV))
    σ_y_prop = dydl * σ_λ
    σ_y = np.sqrt(σ_y_prop ** 2 + σ_A_1 ** 2)

    # Fit pesato finale
    popt, _ = curve_fit(parabola, x_data, y_data, p0=popt, sigma=σ_y, absolute_sigma=True)

    y_fit = parabola(x_data, *popt)
    residuals = (y_data - y_fit) / σ_y
    chi_squared = np.sum(residuals**2)
    dof = len(y_data) - len(popt)
    chi_squared_reduced = chi_squared / dof
    p_value = 1 - chi2.cdf(chi_squared, dof)

    chi_squared_results.append({
        'Tempo (ns)': key,
        'Chi-squared-reduced': chi_squared_reduced,
        'p-value': p_value
    })

# Creazione del DataFrame
chi_squared_results_df = pd.DataFrame(chi_squared_results)
# Stampa della tabella
print(chi_squared_results_df.round(5))

    Tempo (ns)  Chi-squared-reduced  p-value
0            5              0.02713  0.99999
1           12              0.05419  0.99992
2           21              0.05280  0.99993
3           28              0.04926  0.99995
4           36              0.04235  0.99997
5           43              0.01366  1.00000
6           50              0.01326  1.00000
7           60              0.04864  0.99995
8           72              0.00733  1.00000
9           84              0.01546  1.00000
10          97              0.03262  0.99999


# Tempo medio di ingresso della rodamina nella pastiglia

<h3> Funzione per il fit dei picchi

In [13]:
def delta_abs (t, a, b, k):
    return b * (1 - a * np.exp(-k * t))

<h3> Fit esponenziale

In [14]:
# # propagazione errori
# def propagate_errors(t, a, b, k, sigma_a, sigma_b, sigma_k):
#     # Derivate parziali
#     d_delta_abs_da = -b * np.exp(-k * t)
#     d_delta_abs_db = 1 - a * np.exp(-k * t)
#     d_delta_abs_dk = b * a * t * np.exp(-k * t)

#     # Propagazione degli errori
#     sigma_delta_abs = np.sqrt((d_delta_abs_da * sigma_a) ** 2 + (d_delta_abs_db * sigma_b) ** 2 + (d_delta_abs_dk * sigma_k) ** 2)
#     return sigma_delta_abs

# # fit esponenziale

# popt_exp, pcov_exp = curve_fit(delta_abs, tempi[1:], minimi[1:], p0 = [1, 1, 0.01])
# a, b, k = popt_exp

# # Fit esponenziale
# popt, pcov = curve_fit(delta_abs, tempi[1:], minimi[1:], p0=[1, 1, 0.01])
# a, b, k = popt
# err_a, err_b, err_k = np.sqrt(np.diag(pcov))
# # Calcolo degli errori propagati
# sigma_delta_abs = []
# for t in tempi[1:]:
#     sigma = propagate_errors(t, a, b, k, err_a, err_b, err_k)
#     sigma_delta_abs.append(sigma)
# sigma_delta_abs = np.array(sigma_delta_abs)

# value = delta_abs(0, a, b, k)
# error = propagate_errors(0, a, b, k, err_a, err_b, err_k)
# final_value = final_val(value, error)
# final_error = propagate_errors(0, a, b, k, err_a, err_b, err_k)
# # Stampa dei risultati
# print(f"Valore finale: {final_value}")

In [15]:
# Prepara i dati per il curve fitting
t = np.array(list(picchi.keys()))  # Tempi
# Extract the maximum 'A' values from each dataframe in picchi
A = np.array([df['A'].max() for df in picchi.values()])  # Ampiezze

# Stima iniziale dei parametri (a, b, k)
p0 = [1, 0.1, 0.01]

# Curve fitting with increased maxfev and refined initial parameters
popt1, pcov1 = curve_fit(delta_abs, t, A, p0=p0, maxfev=5000)

# perr = np.sqrt(np.diag(pcov))  # Incertezze standard sui parametri
# print(f"Parameters: a = {popt[0]:.4f} ± {perr[0]:.4f}, b = {popt[1]:.4f} ± {perr[1]:.4f}, k = {popt[2]:.4f} ± {perr[2]:.4f}")

In [16]:
# Crea una figura per i picchi e il fit
fig_fit = go.Figure()

t_fit = np.linspace(t.min(), t.max(), 500)  # Intervallo di tempo per il fit
A_fit = delta_abs(t_fit, *popt1)  # Valori del fit

# Aggiungi la curva del fit
fig_fit.add_trace(go.Scatter(
    x=t_fit,
    y=A_fit,
    mode='lines',
    name='Fit',
    line=dict(color = colors[3], width=2),
    showlegend=False
))

# Dati originali (picchi)
picchi_values = np.array([df['A'].max() for df in picchi.values()])
picchi_times = np.array(list(picchi.keys()))

# Aggiungi i picchi
fig_fit.add_trace(go.Scatter(
    x=picchi_times,
    y=picchi_values,
    mode='markers',
    name='Picchi',
    marker=dict(color=colors[10], size=3),
    error_y=dict(
        type='data',
        array=yV_err,
        visible=True,
        color=colors[10],
        thickness=1.5,
        width=3
    ),
    showlegend=False
))

# # Aggiungi gli errori sui picchi (yV_err)
# for idx, key in enumerate(picchi.keys()):
#     picco_error = yV_err[idx]  # Errore sul picco
#     fig_fit.add_trace(go.Scatter(
#         x=[picchi_times[idx], picchi_times[idx]],  # x del picco
#         y=[picchi_values[idx] - picco_error, picchi_values[idx] + picco_error],  # errore sopra e sotto il picco
#         mode='lines',
#         line=dict(color=colors[6], ),
#         showlegend=False
#     ))

fig_fit.update_layout(
    xaxis_title='t (ns)',
    yaxis_title='ΔA',
    legend_title='Legenda',
    width=600,
    template='plotly_white'
)

fig_fit.show()

# Stampa i parametri ottimizzati con gli errori
perr1 = np.sqrt(np.diag(pcov))

# Parametri del fit con errore
print(f"Parametri: a = {popt1[0]:.4f} ± {perr1[0]:.4f}, b = {popt1[1]:.4f} ± {perr1[1]:.4f}, k = {popt1[2]:.4f} ± {perr1[2]:.4f}")
print('\n')
print('Tempo impiegato (in media) dalla rodamina per essere assorbita dalla pastiglia:')
print(f't = ({1/popt1[2]:.4f} ± {perr1[2]/(popt1[2]**2):.4f}) ns')


Parametri: a = 0.9288 ± 0.0002, b = 0.1318 ± 1.0488, k = 0.0170 ± 0.0019


Tempo impiegato (in media) dalla rodamina per essere assorbita dalla pastiglia:
t = (58.9788 ± 6.5742) ns


<h3> Chi quadri e pvalue

In [17]:
#print(yV_err)
res1 = (A - delta_abs(t, *popt1)) / yV_err  # residui normalizzati
chi_squared1 = np.sum(res1 ** 2)
chi_squared_reduced1 = chi_squared1 / (len(A) - len(popt1))  # Chi quadro ridotto
dof1 = len(A) - len(popt1)  # gradi di libertà
p_value1 = 1 - chi2.cdf(chi_squared1, dof1)
print(chi_squared1)
print(f"Chi quadro ridotto: {chi_squared_reduced1:.4f}")
print(f"p-value: {p_value1:.4f}")


15.06864192014291
Chi quadro ridotto: 1.8836
p-value: 0.0578


- Ho tolto il fondo nel fit finale perché ha errore zero, e questa cosa manda in palla il codice
- Fit parabolici perfetti (fin troppo), fit esponenziale traballante
- ERRORI !!11!1!!11   TT